In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [5]:
# 数据预处理：将图像转换为PyTorch张量并标准化
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
# 加载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [11]:
# 定义一个简单的卷积神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
# 检查GPU是否可用，如果可用则使用，否则使用CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [15]:
net = Net().to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# 训练网络
for epoch in range(20): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        inputs, labels = inputs.to(device), labels.to(device)  # 将数据移到GPU上


        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1, 2000] loss: 2.183
[1, 4000] loss: 1.846
[1, 6000] loss: 1.638
[1, 8000] loss: 1.585
[1, 10000] loss: 1.532
[1, 12000] loss: 1.484
[2, 2000] loss: 1.432
[2, 4000] loss: 1.388
[2, 6000] loss: 1.384
[2, 8000] loss: 1.336
[2, 10000] loss: 1.329
[2, 12000] loss: 1.291
[3, 2000] loss: 1.252
[3, 4000] loss: 1.239
[3, 6000] loss: 1.207
[3, 8000] loss: 1.212
[3, 10000] loss: 1.201
[3, 12000] loss: 1.203
[4, 2000] loss: 1.117
[4, 4000] loss: 1.131
[4, 6000] loss: 1.146
[4, 8000] loss: 1.141
[4, 10000] loss: 1.159
[4, 12000] loss: 1.134
[5, 2000] loss: 1.063
[5, 4000] loss: 1.069
[5, 6000] loss: 1.052
[5, 8000] loss: 1.070
[5, 10000] loss: 1.067
[5, 12000] loss: 1.070
[6, 2000] loss: 0.987
[6, 4000] loss: 1.010
[6, 6000] loss: 1.020
[6, 8000] loss: 1.018
[6, 10000] loss: 1.018
[6, 12000] loss: 1.016
[7, 2000] loss: 0.924
[7, 4000] loss: 0.953
[7, 6000] loss: 0.959
[7, 8000] loss: 0.977
[7, 10000] loss: 1.006
[7, 12000] loss: 0.964
[8, 2000] loss: 0.873
[8, 4000] loss: 0.902
[8, 6000] loss: 0.

In [19]:
# 测试网络性能
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total}%')

Accuracy of the network on the 10000 test images: 61%
